In [4]:
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
def parentChatbotSummarize(filepath: str, promptQuestion: str ):
    llm = ChatOpenAI(model = 'gpt-4o')

    loader = JSONLoader(file_path=filepath, jq_schema='.', text_content=False)
    docs=loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap= 200,add_start_index=True)
    splits = text_splitter.split_documents(docs)
    vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding= OpenAIEmbeddings())
    retriever = vectorstore.as_retriever(search_type= 'similarity')

    system_prompt = (
    "You are a nurse explaining to a parent how their premature child is doing in the NICU. You cannot make a diagnoses, just explain briefly."
    "Use the following retrieved context to provide relevant information to the parent. "
    "Speak in a conversational tone, and keep the summary concise."
    "\n\n{context}"
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    
    response = rag_chain.invoke({"input": promptQuestion})

    return response['answer']





    

In [6]:
parentChatbotSummarize('Gurshan_Data.json','Is my baby comfortable?')

<generator object RunnableBindingBase.stream at 0x00000231A4CC1300>